<a href="https://colab.research.google.com/github/misabhishek/gcp-iam-recommender/blob/main/iam_recommender_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you begin


1.  Have a GCP projrect ready. 
3.  [Enable Iam Recommender](https://console.cloud.google.com/flows/enableapi?apiid=recommender.googleapis.com) APIs for the project.

### Provide your credentials to the runtime

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

## Understand GCP IAM Recommender

**Declare the Cloud project ID which will be used throughout this notebook**

In [ ]:
project_id = "Enter-your-project"

**A helper function to execute `gcloud` commands**

In [ ]:
import json
import subprocess
def execute_command(command):
    return json.loads(subprocess.check_output(filter(lambda x: x, command.split(" "))).decode("utf-8"))

In [ ]:
recommender_command = f"""gcloud recommender recommendations list \
                         --location=global \
                         --recommender=google.iam.policy.Recommender \
                         --project={project_id} \
                         --format=json
                        """

In [ ]:
recommendations = execute_command(recommender_command)

In [ ]:
recommendations[7]

### Getting insight for the recommendations

In [ ]:
insight_command = f"""gcloud recommender insights list \
    --project={project_id} \
    --location=global \
    --insight-type=google.iam.policy.Insight \
    --format=json
    """

In [ ]:
insights = execute_command(insight_command)

In [ ]:
insights[0]

# Generate diff view

In [ ]:
recommendation_name = "Enter-the-recommendation-name"

In [ ]:
#@title A helper to generate diff view. It uses IAM roles api also.
import pandas as pd
def generate_diff_view(recommendation_name):
  role_to_permission_command = "gcloud iam roles describe {} --format=json"

  recommendation = [r for r in recommendations if r["name"] == recommendation_name][0]
  insight_name = recommendation["associatedInsights"][0]["insight"]

  added_roles = []
  removed_role = []
  for op in recommendation["content"]["operationGroups"][0]["operations"]:
    if op["action"] == "add":
      added_roles.append(op["pathFilters"]["/iamPolicy/bindings/*/role"])
    if op["action"] == "remove":
      removed_role.append(op["pathFilters"]["/iamPolicy/bindings/*/role"])

  cur_permissions = set(execute_command(
    role_to_permission_command.format(removed_role[0]))["includedPermissions"])

  recommended_permisisons = set() 
  for r in added_roles:
    recommended_permisisons.update(execute_command(
        role_to_permission_command.format(r))["includedPermissions"])
    
  removed_permisisons = cur_permissions - recommended_permisisons
  
  insight = [insight for insight in insights 
             if insight["name"] == insight_name][0]
  used_permissions = set(k["permission"] for k in 
                       insight["content"]["exercisedPermissions"])
  inferred_permissions = set(k["permission"] for k in 
                           insight["content"]["inferredPermissions"])
  
  unused_but_still_common_permissions = (recommended_permisisons - used_permissions 
                                         - inferred_permissions)
  
  types = (["used"] * len(used_permissions) 
         + ["ml-inferred"] * len(inferred_permissions)
         + ["common"] * len(unused_but_still_common_permissions)
         + ["removed"] * len(removed_permisisons))
  
  permissions = [*used_permissions, *inferred_permissions, 
               *unused_but_still_common_permissions, *removed_permisisons]

  return pd.DataFrame({"type": types, "permission": permissions})

In [ ]:
diff_view = generate_diff_view(recommendation_name)

In [ ]:
diff_view

In [ ]:
diff_view["type"].value_counts()